## 14.1 组合查询
&emsp;&emsp;多数SQL 查询只包含从一个或多个表中返回数据的单条SELECT 语句。但是，SQL 也允许执行多个查询（多条SELECT 语句），并将结果作为一个查询结果集返回。这些组合查询通常称为并（union）或复合查询（compound query）。主要有两种情况需要使用组合查询：
- 在一个查询中从不同的表返回结构数据；
- 对一个表执行多个查询，按一个查询返回数据。

>**提示**：任何具有多个WHERE 子句的SELECT 语句都可以作为一个组合查询。

In [1]:
import sqlite3 as sql 
import pandas as pd

def select_sql(query):
    conn = sql.connect('tysql.sqlite')
    cur = conn.cursor()
    df = pd.read_sql(query, con=conn)
    cur.close()
    conn.close()
    print(df)

## 14.2 创建组合查询
&emsp;&emsp;可用UNION 操作符来组合数条SQL 查询。利用UNION，可给出多条SELECT 语句，将它们的结果组合成一个结果集。

### 14.2.1 使用UNION

In [2]:
# 创建UNION 组合多条SELECT 语句
query = '''
SELECT cust_name, cust_contact, cust_email
FROM Customers
WHERE cust_state IN ('IL','IN','MI')
UNION
SELECT cust_name, cust_contact, cust_email
FROM Customers
WHERE cust_name = 'Fun4All';
'''
select_sql(query)

       cust_name        cust_contact             cust_email
0        Fun4All  Denise L. Stephens  dstephens@fun4all.com
1        Fun4All           Jim Jones     jjones@fun4all.com
2  The Toy Store          Kim Howard                   None
3   Village Toys          John Smith  sales@villagetoys.com


In [3]:
# 使用多条WHERE 子句而不是UNION 的相同查询
query = '''
SELECT cust_name, cust_contact, cust_email
FROM Customers
WHERE cust_state IN ('IL','IN','MI')
        OR cust_name = 'Fun4All';
'''
select_sql(query)

       cust_name        cust_contact             cust_email
0   Village Toys          John Smith  sales@villagetoys.com
1        Fun4All           Jim Jones     jjones@fun4all.com
2        Fun4All  Denise L. Stephens  dstephens@fun4all.com
3  The Toy Store          Kim Howard                   None


>**提示**：使用UNION 组合SELECT 语句的数目，SQL 没有标准限制。但是，最好是参考一下具体的DBMS 文档，了解它是否对UNION 能组合的最大语句数目有限制。

>**注意**：理论上讲，从性能上看使用多条WHERE 子句条件还是UNION 应该没有实际的差别。不过我说的是理论上，实践中多数查询优化程序并不能达到理想状态，所以最好测试一下这两种方法，看哪种工作得更好。

### 14.2.2 UNION 规则
- UNION 必须由两条或两条以上的SELECT 语句组成，语句之间用关键字UNION 分隔（因此，如果组合四条SELECT 语句，将要使用三个UNION关键字）；
- UNION 中的每个查询必须包含相同的列、表达式或聚集函数（不过，各个列不需要以相同的次序列出）；
- 列数据类型必须兼容：类型不必完全相同，但必须是DBMS 可以隐含转换的类型（例如，不同的数值类型或不同的日期类型）。

### 14.2.3 包含或取消重复的行
&emsp;&emsp;UNION 从查询结果集中自动去除了重复的行；换句话说，它的行为与一条SELECT 语句中使用多个WHERE 子句条件一样。使用UNION 时，重复的行会被自动取消。

In [4]:
# 使用UNION ALL，返回所有的匹配行
query = '''
SELECT cust_name, cust_contact, cust_email
FROM Customers
WHERE cust_state IN ('IL','IN','MI')
UNION ALL
SELECT cust_name, cust_contact, cust_email
FROM Customers
WHERE cust_name = 'Fun4All';
'''
select_sql(query)

       cust_name        cust_contact             cust_email
0   Village Toys          John Smith  sales@villagetoys.com
1        Fun4All           Jim Jones     jjones@fun4all.com
2  The Toy Store          Kim Howard                   None
3        Fun4All           Jim Jones     jjones@fun4all.com
4        Fun4All  Denise L. Stephens  dstephens@fun4all.com


>**提示**：如果确实需要每个条件的匹配行全部出现（包括重复行），就必须使用UNION ALL，而不是WHERE。

### 14.2.4 对组合查询结果进行排序
&emsp;&emsp;在用UNION 组合查询时，只能使用一条ORDER BY 子句，它必须位于最后一条SELECT 语句之后。

In [5]:
# 对UNION 返回的结果进行排序
query = '''
SELECT cust_name, cust_contact, cust_email
FROM Customers
WHERE cust_state IN ('IL','IN','MI')
UNION
SELECT cust_name, cust_contact, cust_email
FROM Customers
WHERE cust_name = 'Fun4All'
ORDER BY cust_name, cust_contact;
'''
select_sql(query)

       cust_name        cust_contact             cust_email
0        Fun4All  Denise L. Stephens  dstephens@fun4all.com
1        Fun4All           Jim Jones     jjones@fun4all.com
2  The Toy Store          Kim Howard                   None
3   Village Toys          John Smith  sales@villagetoys.com


>**说明**：某些DBMS 还支持另外两种UNION：EXCEPT（有时称为MINUS）可用来检索只在第一个表中存在而在第二个表中不存在的行； 而INTERSECT 可用来检索两个表中都存在的行。实际上，这些UNION 很少使用，因为相同的结果可利用联结得到。

>**提示**：UNION 在需要组合多个表的数据时也很有用，即使是有不匹配列名的表，在这种情况下，可以将UNION 与别名组合，检索一个结果集。

## 14.3 小结
&emsp;&emsp;本章学习了如何用UNION 操作符来组合SELECT 语句。利用UNION，可以把多条查询的结果作为一条组合查询返回，不管结果中有无重复。使用UNION 可极大地简化复杂的WHERE 子句，简化从多个表中检索数据的工作。